In [1]:
import spiceypy
# Allows us to convert to ephemeris time

In [4]:
import datetime
date_today = datetime.datetime.today()

date_today = date_today.strftime("%Y-%m-%dT00:00:00")

In [5]:
print(date_today)

2024-09-28T00:00:00


In [6]:
et_today_midnight = spiceypy.utc2et(date_today)
print(et_today_midnight)

SpiceNOLEAPSECONDS: 
================================================================================

Toolkit version: CSPICE_N0067

SPICE(NOLEAPSECONDS) --

The variable that points to the leapseconds (DELTET/DELTA_AT) could not be located in the kernel pool.  It is likely that the leapseconds kernel has not been loaded.

utc2et_c --> UTC2ET --> TTRANS

================================================================================